In [5]:
import numpy as np
import pandas as pd

from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split

In [6]:
# read original data from 
data = pd.read_csv('../dataframes/DF_prest.csv', index_col=0)

# setup 'docs' for use with Tokenizer
def nt_seq_doc(nt_sequence):
    if 'GACAAGCTTGCGGCCGCA' not in nt_sequence:
        return None
    true_nt = nt_sequence.split('GACAAGCTTGCGGCCGCA')[1]
    if len(true_nt) % 3 != 0:
        return None
    return ' '.join([true_nt[i:i+3] 
                     for i in range(0, len(true_nt), 3)])
# split quantiles
def assign_class(conc):
    if conc <= low_cut:
        return 0
    elif conc >= high_cut:
        return 1
    return

data['nt_seq_doc'] = data['nt_seq'].apply(nt_seq_doc)
data = data[pd.notnull(data['nt_seq_doc'])]

# identify high and low classes by conc_cf quantiles
low_cut = data['conc_cf'].quantile(0.25)
high_cut = data['conc_cf'].quantile(0.75)

data['class'] = data['conc_cf'].apply(assign_class)
data = data[pd.notnull(data['class'])]
# check shape
print('data shape: ', data.shape)

data shape:  (22364, 8)


In [22]:
# define sequence documents
docs = list(data['nt_seq_doc'])
# create the tokenizer
t = Tokenizer()
# fit the tokenizer on the documents
t.fit_on_texts(docs)
# summarize what was learned
print(t.word_counts)
print(t.document_count)
print(t.word_index)
print(t.word_docs)
# integer encode documents
X = t.texts_to_sequences(docs)
y = data['class'].values

# create test-train split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

# fix random seed for reproducibility
np.random.seed(7)

# load the dataset but only keep the top n words, zero the rest
top_words = len(t.word_index) + 1

# truncate and pad input sequences
seq_lengths = [len(seq) for seq in X]
max_seq_length = max(seq_lengths)
X_train = sequence.pad_sequences(X_train, maxlen=max_seq_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_seq_length)

OrderedDict([('acc', 31684), ('tac', 26551), ('tat', 22043), ('gcc', 43066), ('tgg', 21420), ('aag', 54456), ('cat', 19009), ('gag', 67563), ('ctg', 62966), ('ggc', 56405), ('tct', 27655), ('tgc', 19420), ('ccg', 10715), ('ttg', 22138), ('ccc', 55231), ('agg', 20309), ('gtg', 44810), ('atg', 34356), ('cta', 12299), ('cag', 59327), ('gct', 30054), ('tcc', 31193), ('gca', 27081), ('gcg', 32244), ('tca', 22471), ('agc', 35014), ('cct', 30135), ('gga', 27189), ('gtc', 23247), ('tta', 13479), ('cgg', 18621), ('atc', 31254), ('aac', 33661), ('aca', 26639), ('gta', 12449), ('cgc', 37768), ('ttc', 32044), ('aaa', 41705), ('act', 22748), ('aat', 31134), ('cga', 10633), ('ctc', 30427), ('cca', 30088), ('ggg', 25603), ('gtt', 18774), ('gaa', 53144), ('att', 25675), ('agt', 22614), ('ggt', 17735), ('taa', 22344), ('cac', 48286), ('tga', 22350), ('aga', 20896), ('ttt', 28701), ('acg', 10281), ('cgt', 7504), ('caa', 22951), ('ctt', 22010), ('tgt', 16982), ('ata', 13093), ('gac', 44691), ('gat', 3964

# Embed codons in 8 dims. Learn w/ single Long Short Term Memory Layer

In [56]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

# create the model
embedding_vecor_length = 8
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_seq_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, epochs=3, batch_size=100)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, 181, 8)            520       
_________________________________________________________________
lstm_12 (LSTM)               (None, 100)               43600     
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 101       
Total params: 44,221
Trainable params: 44,221
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
15654/15654 [==============================] - 27s 2ms/step - loss: 0.6907 - acc: 0.5378
Epoch 2/3
15654/15654 [==============================] - 26s 2ms/step - loss: 0.6794 - acc: 0.5719
Epoch 3/3
15654/15654 [==============================] - 26s 2ms/step - loss: 0.6582 - acc: 0.6122
Accuracy: 62.00%


# Add dropout to LSTM network

In [47]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

# create the model
embedding_vecor_length = 8
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_seq_length))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, epochs=3, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 181, 8)            520       
_________________________________________________________________
dropout_3 (Dropout)          (None, 181, 8)            0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 100)               43600     
_________________________________________________________________
dropout_4 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 101       
Total params: 44,221
Trainable params: 44,221
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
15654/15654 [==============================] - 37s 2ms/step - loss: 0.6924 - acc: 0.5202
Epoch 2/3
15654/15654 [===========

# Hook LSTM to basic CNN

In [48]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

# create the model
embedding_vecor_length = 8
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_seq_length))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, epochs=3, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 181, 8)            520       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 181, 32)           800       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 90, 32)            0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 101       
Total params: 54,621
Trainable params: 54,621
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
15654/15654 [===========================

In [49]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

# create the model
embedding_vecor_length = 8
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_seq_length))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(Conv1D(filters=32, kernel_size=10, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, epochs=3, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 181, 8)            520       
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 181, 32)           800       
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 181, 32)           10272     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 90, 32)            0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 101       
Total params: 64,893
Trainable params: 64,893
Non-trainable params: 0
_________________________________________________________________
None
E

In [55]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence


# create the model
embedding_vecor_length = 8
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_seq_length))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, epochs=10, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 181, 8)            520       
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 181, 32)           800       
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 181, 32)           3104      
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 90, 32)            0         
_________________________________________________________________
lstm_11 (LSTM)               (None, 100)               53200     
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 101       
Total params: 57,725
Trainable params: 57,725
Non-trainable params: 0
_________________________________________________________________
None
E